In [1]:
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from data_helper import ImageGenerator, get_train_matrices, get_test_matrices, ValidGenerator
from sklearn.cross_validation import train_test_split

Using TensorFlow backend.
/home/westside/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
img_size = (224, 224)
output_size = 17

In [4]:
x_train, y_train, labels_map = get_train_matrices("../data/train_v2.csv", "../data/train-jpg", img_size)
X_train, X_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2)
flow = ImageGenerator().get_train_generator(X_train, y_train)
validation_generator = ValidGenerator().get_valid_generator(X_valid, y_valid)

100%|██████████| 40479/40479 [02:22<00:00, 284.40it/s]


In [ ]:
resnet = ResNet50(include_top=False, weights='imagenet')
x = resnet.output

x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

predictions = Dense(output_size, activation='sigmoid')(x)

model = Model(input=resnet.input, output=predictions)

for layer in resnet.layers:
    layer.trainable = False

epochs = [5, 5]
learning_rates = [0.001, 0.0001]

for e, lr in zip(epochs, learning_rates):
    model.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit_generator(flow, steps_per_epoch=260, nb_epoch=e, validation_data=validation_generator, validation_steps=63)

for layer in resnet.layers[-10:]:
    layer.trainable = True

epochs = [100, 30]
learning_rates = [0.0001, 0.00001]

for e, lr in zip(epochs, learning_rates):
    model.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit_generator(flow, steps_per_epoch=260, nb_epoch=e, validation_data=validation_generator, validation_steps=63)

/home/westside/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  
/home/westside/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., steps_per_epoch=260, validation_data=<keras.pre..., validation_steps=63, epochs=5)`


Epoch 1/5
260/260 [==============================] - 256s - loss: 0.1741 - acc: 0.9337 - val_loss: 0.4217 - val_acc: 0.8134
Epoch 2/5
260/260 [==============================] - 248s - loss: 0.1447 - acc: 0.9451 - val_loss: 0.4052 - val_acc: 0.8416
Epoch 3/5
260/260 [==============================] - 248s - loss: 0.1382 - acc: 0.9474 - val_loss: 0.3617 - val_acc: 0.8532
Epoch 4/5
260/260 [==============================] - 248s - loss: 0.1360 - acc: 0.9482 - val_loss: 0.1425 - val_acc: 0.9435
Epoch 5/5
115/260 [============>.................] - ETA: 106s - loss: 0.1343 - acc: 0.9487

resnet